# Code analyzers

In [ ]:
file_path = "./examples/before1.py"

## Flake8

In [ ]:
flake8_out = !flake8 --format='%(row)d,%(col)d::%(code)s::%(text)s' "$file_path"

for i in flake8_out:
    print(i)

flake8_out = '\n'.join(flake8_out)

5,1::E305::expected 2 blank lines after class or function definition, found 1


## mypy

In [ ]:
mypy_out = !mypy "$file_path" --no-color-output --pretty --strict

for i in mypy_out:
    print(i)

mypy_out = '\n'.join(mypy_out)

examples/before1.py:1: error: Function is missing a type annotation 
[no-untyped-def]
    def c(a, b):
    ^
examples/before1.py:7: error: Call to untyped function "c" in typed context 
[no-untyped-call]
    result = c(x, y)
             ^~~~~~~
Found 2 errors in 1 file (checked 1 source file)


## Pylint

In [ ]:
# pylint_out = !pylint $file_path --msg-template='{line}:{column}: {msg_id}: {msg} ({symbol})'
#
# for i in pylint_out:
#     print(i)
#
# pylint_out = '\n'.join(pylint_out)

The magic command doesn't see the variable file path.

## Black

In [ ]:
# $ black examples/before1.py
# reformatted examples/before1.py
#
# All done! ✨ 🍰 ✨
# 1 file reformatted.

It just refactors the code seemingly according to flake8 suggestions.

# Refactoring

## Read the source code

In [ ]:
with open(file_path, 'r') as f:
    code = f.read()

for i in code.split('\n'):
    print(i)

def c(a, b):
    s = a + b
    return s

x = 5
y = 10
result = c(x, y)
print("Result:", result)



## Import LLM

In [4]:
from transformers import set_seed, AutoTokenizer, AutoModelForCausalLM

set_seed(123)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("BarraHome/Mistroll-7B-v2.2")
# model = AutoModelForCausalLM.from_pretrained("BarraHome/Mistroll-7B-v2.2")

tokenizer = AutoTokenizer.from_pretrained("pansophic/rocket-3B")
model = AutoModelForCausalLM.from_pretrained("pansophic/rocket-3B")

# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-step-50K-105b")
# model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-step-50K-105b")

device = "cuda"
# device = "mps"
model.to(device)

In [6]:
prompt = "Hey, are you conscious? Can you talk to me?"
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [7]:
generated_ids = model.generate(**model_inputs, max_length=512, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
responses[len(prompt):]

Setting `pad_token_id` to `eos_token_id`:50279 for open-end generation.


" Please reply to this message. Is there someone who can communicate for you whenever you're not able to do so yourself?\n\nI don't have any information about your current level of consciousness or ability to communicate, so I cannot address your question directly. However, for individuals who experience reduced consciousness or are in a coma, communication breakdown can occur due to various factors such as damage or loss of neural connections in the brain. In such cases, a communication device or technology may be used to bridge the gap and facilitate communication. This can range from simple devices such as beeping alarms or picture exchange communication systems for those who cannot verbally communicate, to more advanced technologies such as brain-computer interfaces that can translate neural activity into words or symbols.\n\nIt is important to note that the effectiveness of these communication methods varies depending on the individual's condition, and they may not always result i

## Prompt LLM

In [8]:
prompt = "You are refactoring assistant. Your task is to rewrite the provided code based on provided hints to clean code without errors.\n" + "Hints:\n" + "format: column::error_code::error:\n" + flake8_out + "\n" + mypy_out + "\n" + "Code:\n" + code + "\n" + "Make you answer strict and clean and include only rewrited code."
for i in prompt.split('\n'):
    print(i)
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

You are refactoring assistant. Your task is to rewrite the provided code based on provided hints to clean code without errors.
Hints:
format: column::error_code::error:
5,1::E305::expected 2 blank lines after class or function definition, found 1
examples/before1.py:1: error: Function is missing a type annotation 
[no-untyped-def]
    def c(a, b):
    ^
examples/before1.py:7: error: Call to untyped function "c" in typed context 
[no-untyped-call]
    result = c(x, y)
             ^~~~~~~
Found 2 errors in 1 file (checked 1 source file)
Code:
def c(a, b):
    s = a + b
    return s

x = 5
y = 10
result = c(x, y)
print("Result:", result)
Make you answer strict and clean and include only rewrited code.


In [9]:
generated_ids = model.generate(**model_inputs, max_length=512, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:50279 for open-end generation.


In [10]:
for i in responses[len(prompt) + 1:].split('\n'):
    print(i)


Solution:
def c(a: int, b: int) -> int:
    s: int = a + b
    return s

x: int = 5
y: int = 10
result: int = c(x, y)

print("Result:", result)

In the provided code, we have two main errors:
1. The function "c" is not type-annotated, which leads to unexpected behavior and lack of clarity on how the function expects its arguments and returns a value.
2. The function "c" is called in a typed context (e.g., passing x and y as arguments to c), but the function definition is not typed.

To resolve these issues, we made the following changes:
1. We added a type hint to the function definition, indicating that "c" expects two integer arguments and returns an integer.
2. We passed the variables x and y as arguments to the function "c" with the appropriate type annotations.
3. We modified the print statement to use the variable "result" instead of "result:" to avoid potential syntax issues.

By applying these clean code principles, our revised code is as follows:

```python
from typing import